# Exercise 9.01: Training an ANN for Sequential Data - Nvidia Stock Prediction

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
import io
data = pd.read_csv("NVDA.csv")

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
# Split Training data
data_training = data[data['Date']<'2019-01-01'].copy()
data_training

In [ ]:
# Split Testing data
data_test = data[data['Date']>='2019-01-01'].copy()
data_test

In [ ]:
training_data = data_training.drop\
                (['Date', 'Adj Close'], axis = 1)
training_data.head()

In [ ]:
scaler = MinMaxScaler()
training_data = scaler.fit_transform(training_data)
training_data

In [ ]:
X_train = []
y_train = []

In [ ]:
training_data.shape[0]

In [ ]:
for i in range(60, training_data.shape[0]):
  X_train.append(training_data[i-60:i])
  y_train.append(training_data[i, 0])

In [ ]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_old_shape = X_train.shape
X_train = X_train.reshape(X_old_shape[0], \
                          X_old_shape[1]*X_old_shape[2]) 
X_train.shape


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout

In [ ]:
regressor_ann = Sequential()
 
regressor_ann.add(Input(shape = (300,)))
regressor_ann.add(Dense(units = 512, activation = 'relu'))
regressor_ann.add(Dropout(0.2))
 
regressor_ann.add(Dense(units = 128, activation = 'relu'))
regressor_ann.add(Dropout(0.3))
 
regressor_ann.add(Dense(units = 64, activation = 'relu'))
regressor_ann.add(Dropout(0.4))
 
regressor_ann.add(Dense(units = 16, activation = 'relu'))
regressor_ann.add(Dropout(0.5))
 
regressor_ann.add(Dense(units = 1))

In [ ]:
regressor_ann.summary()

In [ ]:
regressor_ann.compile(optimizer='adam', \
                      loss = 'mean_squared_error')

In [ ]:
regressor_ann.fit(X_train, y_train, epochs=10, batch_size=32)

In [ ]:
## Now we will test and predict stock price

## Prepare test dataset
data_test.head()

In [ ]:
data_training.tail(60)

In [ ]:
past_60_days = data_training.tail(60)


In [ ]:
df = past_60_days.append(data_test, ignore_index = True)
df = df.drop(['Date', 'Adj Close'], axis = 1)
df.head()

In [ ]:
inputs = scaler.transform(df)

In [ ]:
X_test = []
y_test = []

for i in range(60, inputs.shape[0]):
  X_test.append(inputs[i-60:i])
  y_test.append(inputs[i, 0])

In [ ]:
X_test, y_test = np.array(X_test), np.array(y_test)
 
X_old_shape = X_test.shape
X_test = X_test.reshape(X_old_shape[0], \
                        X_old_shape[1] * X_old_shape[2])
 
X_test.shape, y_test.shape

In [ ]:
y_pred = regressor_ann.predict(X_test)

In [ ]:
scaler.scale_

In [ ]:
scale = 1/3.70274364e-03
scale

In [ ]:
y_pred = y_pred*scale
y_test = y_test*scale

In [ ]:
plt.figure(figsize=(14,5))
plt.plot(y_test, color = 'black', label = "Real NVDA Stock Price")
plt.plot(y_pred, color = 'gray', label = 'Predicted NVDA Stock Price')
plt.title('NVDA Stock Price Prediction')
plt.xlabel('time')
plt.ylabel('NVDA Stock Price')
plt.legend()
plt.show()
